In [21]:
# Compute Black-Scholes option value using a binomial tree
# European case
# use loops

import math as m
import numpy as np

S0 = 100     # S0 - current stock price
K = 100      # K - strike
T = 1.0      # T - expiry time
r = .04      # r - interest rate
sigma = .32  # sigma - volatility
opttype = 0  # opttype - 0 for a call, otherwise a put
Nsteps = 500 # Nsteps - number of timesteps


def Eurpayoff(S0, K, T, r, sigma, opttype, Nsteps):
    # generate payoff of European call/put option
    delt = T/Nsteps
    # tree parameters
    u = m.exp(sigma * m.sqrt(delt))
    d = 1/u
    a = m.exp(r * delt)
    p = (a - d)/(u - d)
    
    # pay off at t = T
    W = S0 * u ** np.arange(Nsteps + 1) * d ** np.arange(Nsteps, -1, -1)
    
    #vector operations
    if (opttype == 0):
        W = np.maximum(W - K, 0);  # call case
    else:
        W = np.maximum(K - W, 0);  # put case
        
    for j in np.arange(Nsteps, 0, -1):
        W = np.exp(-r * delt) * (p * W[1:j+1] + (1 - p) * W[0:j]) 
        # European lattice algorithm
        value = W[0]
    return value


In [22]:
def Amepayoff_call(S0, K, T, r, sigma, Nsteps): 
    # generate payoff of American call option
    
    delt = T/Nsteps
    # tree parameters
    u = m.exp(sigma * m.sqrt(delt))
    d = 1/u
    a = m.exp(r * delt)
    p = (a - d)/(u - d)
    
    # pay off at t = T
    W = S0 * u ** np.arange(Nsteps + 1) * d ** np.arange(Nsteps, -1, -1)
    W = np.maximum(W - K, 0)  # call case
    currprice = W
    
    for j in np.arange(Nsteps, 0, -1):
        W = np.exp(-r * delt) * (p * W[1:j+1] + (1 - p) * W[0:j]) 
        # American lattice algorithm
        currprice = u * currprice[0:j] 
        W = np.maximum(W, np.maximum(currprice - K, 0))
        value = W[0]
    return value


In [30]:
def Amepayoff_put(S0, K, T, r, sigma, Nsteps): 
    # generate payoff of American put option
    
    delt = T/Nsteps
    # tree parameters
    u = m.exp(sigma * m.sqrt(delt))
    d = 1/u
    a = m.exp(r * delt)
    p = (a - d)/(u - d)
    
    # pay off at t = T
    W = S0 * u ** np.arange(Nsteps + 1) * d ** np.arange(Nsteps, -1, -1)
    currprice = W
    W = np.maximum(K - W, 0)  # put case
    
    for j in np.arange(Nsteps, 0, -1):
        W = np.exp(-r * delt) * (p * W[1:j+1] + (1 - p) * W[0:j]) 
        # American lattice algorithm
        currprice = u * currprice[0:j] 
        W = np.maximum(W, np.maximum(K - currprice, 0))
        value = W[0]
    return value


In [31]:
Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 500) # payoff of European call options

14.513319919417471

In [32]:
Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 500) # payoff of European put options

10.592263834646982

In [33]:
Amepayoff_call(100, 100, 1.0, 0.04, 0.32, 500) # payoff of American call options

14.513319919417471

In [34]:
Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 500) # payoff of American put options

10.991823139809181

In [36]:
from scipy.stats import norm

# generating a Black-Scholes Model 
def blsprice(Price, Strike ,Rate, Time, Volatility):
    denomi = Volatility * m.sqrt(Time)  
    # calculating volatility times the square root of time
    numera = m.log(Price / Strike) + (Rate + 0.5 * Volatility ** 2 ) * Time
    d1 = numera / denomi
    d2 = d1 - denomi
    cdf1 = norm.cdf(d1) # calculating N(d1)
    cdf2 = norm.cdf(d2) # calculating N(d2)
    
    factor = m.exp(- Rate * Time)
    
    call = Price * cdf1 - Strike * factor * cdf2 # calculating the call option price 
    put = Strike * factor * (1 - cdf2) - Price * (1 - cdf1)
    # calculating the put option price 
    return call, put

def blsdelta(Price, Strike , Rate, Time, Volatility):
    denomi = Volatility * m.sqrt(Time)  
    # calculating volatility times the square root of time
    numera = m.log(Price / Strike) + (Rate + 0.5 * Volatility ** 2 ) * Time
    d1 = numera / denomi
    c_delt = norm.cdf(d1)
    delta_call = c_delt
    delta_put = c_delt - 1
    return delta_call, delta_put


In [37]:
blsprice(100, 100, 0.04, 1.0, 0.32) # generating the call/put option prices

(14.519595451443337, 10.598539366675652)

In [38]:
blsdelta(100, 100, 0.04, 1.0, 0.32) # generating the call/put delta

(0.6121779290374987, -0.3878220709625013)

In [48]:
import tabulate as tab

# generating convergence table for European call options

# generating column of values
v1 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 500)
v2 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 1000)
v3 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 2000)
v4 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 4000)
v5 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 0, 8000)

# generating column of changes
c2_1 = v2 - v1
c3_2 = v3 - v2
c4_3 = v4 - v3
c5_4 = v5 - v4

# generating column of ratio
r3 = c2_1 / c3_2
r4 = c3_2 / c4_3
r5 = c4_3 / c5_4

# generating column of delt_t
dt1 = T / 500
dt2 = T / 1000
dt3 = T / 2000
dt4 = T / 4000
dt5 = T / 8000

# generating the exact price of the call option
exact = blsprice(100, 100, 0.04, 1.0, 0.32)[0]

names = ["Δt", "Value", "Change", "Ratio"]
call_convtab = [[dt1, v1],
                [dt2, v2, c2_1],
                [dt3, v3, c3_2, r3],
                [dt4, v4, c4_3, r4],
                [dt5, v5, c5_4, r5],
                ["exact", exact]]
print(tab.tabulate(call_convtab, headers = names))


Δt          Value       Change    Ratio
--------  -------  -----------  -------
0.002     14.5133
0.001     14.5165  0.00313737
0.0005    14.518   0.00156898   1.99962
0.00025   14.5188  0.000784564  1.99981
0.000125  14.5192  0.000392301  1.99991
exact     14.5196


In [49]:
# generating convergence table for European put options

# generating column of values
pv1 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 500)
pv2 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 1000)
pv3 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 2000)
pv4 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 4000)
pv5 = Eurpayoff(100, 100, 1.0, 0.04, 0.32, 1, 8000)

# generating column of changes
pc2_1 = pv2 - pv1
pc3_2 = pv3 - pv2
pc4_3 = pv4 - pv3
pc5_4 = pv5 - pv4

# generating column of ratio
pr3 = pc2_1 / pc3_2
pr4 = pc3_2 / pc4_3
pr5 = pc4_3 / pc5_4

# generating column of delt_t
dt1 = T / 500
dt2 = T / 1000
dt3 = T / 2000
dt4 = T / 4000
dt5 = T / 8000

# generating the exact price of the put option
pexact = blsprice(100, 100, 0.04, 1.0, 0.32)[1]

names = ["Δt", "Value", "Change", "Ratio"]
put_convtab = [[dt1, pv1],
                [dt2, pv2, pc2_1],
                [dt3, pv3, pc3_2, pr3],
                [dt4, pv4, pc4_3, pr4],
                [dt5, pv5, pc5_4, pr5],
                ["exact", pexact]]
print(tab.tabulate(put_convtab, headers = names))


Δt          Value       Change    Ratio
--------  -------  -----------  -------
0.002     10.5923
0.001     10.5954  0.00313737
0.0005    10.597   0.00156898   1.99962
0.00025   10.5978  0.000784564  1.99981
0.000125  10.5981  0.000392301  1.99991
exact     10.5985


In [46]:
# generating convergence table for American put options

# generating column of values
value1 = Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 500)
value2 = Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 1000)
value3 = Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 2000)
value4 = Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 4000)
value5 = Amepayoff_put(100, 100, 1.0, 0.04, 0.32, 8000)

# generating column of changes
change2_1 = value2 - value1
change3_2 = value3 - value2
change4_3 = value4 - value3
change5_4 = value5 - value4

# generating column of ratio
ratio3 = change2_1 / change3_2
ratio4 = change3_2 / change4_3
ratio5 = change4_3 / change5_4

# generating column of delt_t
dt1 = T / 500
dt2 = T / 1000
dt3 = T / 2000
dt4 = T / 4000
dt5 = T / 8000

names = ["Δt", "Value", "Change", "Ratio"]
convtab = [[dt1, value1],
           [dt2, value2, change2_1],
           [dt3, value3, change3_2, ratio3],
           [dt4, value4, change4_3, ratio4],
           [dt5, value5, change5_4, ratio5]]
print(tab.tabulate(convtab, headers = names))


      Δt    Value       Change    Ratio
--------  -------  -----------  -------
0.002     10.9918
0.001     10.9935  0.00163496
0.0005    10.9943  0.000822012  1.98897
0.00025   10.9947  0.000404171  2.03382
0.000125  10.9949  0.000202384  1.99705


We see from the three tables that for the cases of European call/put and American put options, the ratios are approaching 2, so the order of accuracy is 1, which agrees with the theory above.